# Quantum Kernel Alignment with Qiskit Runtime

#### Classification with Support Vector Machines
Classification problems are widespread in machine learning applications. A common method for tackling these problems is the support vector machine (SVM) [1,2]. This supervised learning algorithm uses labeled training samples to find an optimal separating hyperplane between data classes in order to predict the labels of new data points. Often, data is not linearly separable in the original space. In these cases, the kernel trick is used to implicitly encode a transformation of the data into a higher-dimensional feature space through the inner product between pairs of data points, where the data may become separable.

#### Quantum Kernels
Quantum computers can be used to encode classical data in a quantum-enhanced feature space. In 2019, IBM introduced a quantum algorithm called the quantum kernel estimator (QKE), which uses quantum circuits to evaluate inner products between data points in the quantum feature space [3]. More recently, IBM proved, on certain learning problems, quantum kernels can offer superpolynomial speedups over any classical learner [4]. This means that quantum kernels can someday offer quantum advantage on suitable problems. 

#### Learning Quantum Kernels for a Dataset
In practice, SVMs require a choice of the kernel function. Sometimes, symmetries in the data can inform this selection, other times it is chosen in an ad hoc way. Kernel alignment is one approach to learning a kernel on a given dataset by iteratively adapting it to have high similarity to a target kernel informed from the underlying data distribution [5]. As a result, the SVM with an aligned kernel will likely generalize better to new data than with an unaligned kernel. Using this concept, we introduce quantum kernel alignment (QKA), which is a new algorithm for learning a quantum kernel that maximizes the alignment while converging to the maximum SVM margin [6]. 

#### Speeding up Quantum Algorithms with Qiskit Runtime
QKA is an iterative quantum-classical algorithm, in which quantum hardware executes parametrized quantum circuits (which are used to compute the quantum kernel matrices according to the QKE), while a classical optimizer tunes the parameters of the circuits to maximize the alignment. Iterative algorithms of this type can be slow due to latency between the quantum and classical calculations. Qiskit Runtime is a new framework (?) that can speed up iterative algorithms like QKA by pushing the classical computations into the cloud near (?) the quantum hardware. <span style="color:blue">TODO: add more details here on how runtime can help QKA run faster...</span>.

#### Trying This Out on a Specific Learning Problem
<span style="color:blue">TODO: discuss the specific problem we are solving... In this tutorial, we define a particular learning problem we call ...</span>.



[1] B. E. Boser, I. M. Guyon,  and V. N. Vapnik, Proceedings of the Fifth Annual Workshop on Computational Learning Theory, COLT ’92 (Association for Computing Machinery, New York, NY, USA, 1992) pp. 144-152 [link](https://doi.org/10.1145/130385.130401) <br>
[2] V. Vapnik, The Nature of Statistical Learning Theory, Information Science and Statistics (Springer New York, 2013) [link](https://books.google.com/books?id=EqgACAAAQBAJ) <br>
[3] V. Havlíček, A. D. Córcoles, K. Temme, A. W. Harrow, A. Kandala, J. M. Chow, and J. M. Gambetta, Nature 567, 209-212 (2019) [link](https://doi.org/10.1038/s41586-019-0980-2) <br>
[4] Y. Liu, S. Arunachalam, and K. Temme, arXiv:2010.02174 (2020) [link](https://arxiv.org/abs/2010.02174) <br>
[5] N. Cristianini, J. Shawe-taylor, A. Elisseeff, and J. Kandola, Advances in Neural Information Processing Systems 14 (2001) [link](https://proceedings.neurips.cc/paper/2001/file/1f71e393b3809197ed66df836fe833e5-Paper.pdf) <br>
[6] Our ref - forthcoming...

# Load your IBMQ account and get the quantum backend

We'll be using the 27-qubit backend `ibmq_montreal` for this tutorial.

In [1]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')
backend = provider.get_backend('ibmq_montreal')

# Invoke Quantum Kernel Alignment program

Before running the Runtime Program for QKA, we'll need to prepare the dataset and configure the settings for the algorithm.

## 1. Prepare the dataset

First, we load the dataset from the `csv` file and then extract the training the testing samples.

In [2]:
import numpy as np
import pandas as pd

# load the dataset

df = pd.read_csv('/Users/jen/Q/code/quantum_kernel_data/dataset.csv',sep = ',', header = None)
data = df.values
num_features = np.shape(dat)[1]-1 # feature dimension (twice the qubit number) is determined by dataset

# specify the training and testing sets

num_train = 4 # number of training samples per class
num_test = 5  # number of test samples per class

x_train = data[:2*num_train, :-1]
y_train = data[:2*num_train, -1]

x_test = data[2*num_train:2*(num_train+num_test), :-1]
y_test = data[2*num_train:2*(num_train+num_test), -1]

## 2. Configure the settings for QKA

In [3]:
from qiskit import Aer
from qka.featuremaps import FeatureMapACME

# Set the parameters:
C = 1          # SVM soft-margin penalty
maxiters = 1  # number of SPSA iterations

entangler_map=[[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]] # entanglers in the feature map
initial_layout = [9, 8, 11, 14, 16, 19, 22, 25, 24, 23] # mapping virtual qubits to physical qubits on the chip

# Define the feature map and its initial parameters:
initial_point = [0.1] # np.pi/2 should yield 100% test accuracy
fm = FeatureMapACME(feature_dimension=num_features, entangler_map=entangler_map)

## 3. Set up and run the Runtime Program

There are two main components we'll need to set up for our Runtime Program: `inputs` (the feature map, labeled dataset, optimizer settings, etc) and `options` (the backend). Once these are specified, we'll run the Program on the quantum backend.

In [9]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}\n")

In [10]:
# Call QKA runtime program.
runtime_inputs = {
    'feature_map': fm,
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_point,
    'maxiters': maxiters,
    'C': C,
    'initial_layout': initial_layout
}
options = {'backend_name': backend.name()}
job = provider.runtime.run(program_id="quantum-kernel-alignment",
                              options=options,
                              inputs=runtime_inputs,
                              callback=interim_result_callback,
                              )
print(job.job_id())

c24v0dggh40gub5h9bo0


runtime_job._start_websocket_client:WARNING:2021-04-28 19:31:18,217: An error occurred while streaming results from the server for job c24v0dggh40gub5h9bo0:
Traceback (most recent call last):
  File "/Users/jen/opt/anaconda3/envs/qiskit0.25.0python3.7/lib/python3.7/site-packages/qiskit/providers/ibmq/api/clients/runtime_ws.py", line 66, in _connect
    websocket = await connect(url, extra_headers=self._header)
  File "/Users/jen/opt/anaconda3/envs/qiskit0.25.0python3.7/lib/python3.7/site-packages/websockets/client.py", line 547, in __await_impl__
    extra_headers=protocol.extra_headers,
  File "/Users/jen/opt/anaconda3/envs/qiskit0.25.0python3.7/lib/python3.7/site-packages/websockets/client.py", line 296, in handshake
    raise InvalidStatusCode(status_code)
websockets.exceptions.InvalidStatusCode: server rejected WebSocket connection: HTTP 404

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jen/opt/anaconda3/env

## 4. Retrieve and display the results of the Runtime Program

The output of QKA is the set of aligned kernel parameters and the aligned kernel matrix.

In [11]:
result = job.result()

print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

final result:
{'aligned_kernel_parameters': array([0.09268847]), 'aligned_kernel_matrix': array([[1.        , 0.01049805, 0.05932617, 0.00524902, 0.04125977,
        0.11291504, 0.03869629, 0.03442383, 0.01159668, 0.0637207 ],
       [0.01049805, 1.        , 0.00744629, 0.07568359, 0.00524902,
        0.01696777, 0.00793457, 0.00524902, 0.00256348, 0.00720215],
       [0.05932617, 0.00744629, 1.        , 0.01538086, 0.01489258,
        0.04541016, 0.08691406, 0.0123291 , 0.04333496, 0.02124023],
       [0.00524902, 0.07568359, 0.01538086, 1.        , 0.00134277,
        0.00488281, 0.01330566, 0.00195312, 0.00292969, 0.00256348],
       [0.04125977, 0.00524902, 0.01489258, 0.00134277, 1.        ,
        0.03918457, 0.01269531, 0.01428223, 0.00427246, 0.01928711],
       [0.11291504, 0.01696777, 0.04541016, 0.00488281, 0.03918457,
        1.        , 0.05310059, 0.04541016, 0.01953125, 0.06933594],
       [0.03869629, 0.00793457, 0.08691406, 0.01330566, 0.01269531,
        0.05310059, 

# Use the results of the Runtime Program to test the SVM on new data

With the aligned kernel and its parameters in hand, we can use the `sklearn` package to train the SVM and then predict labels of new test points. A second kernel matrix built from the test points is needed for the SVM decision function, so we'll invoke the Circuit Runner Runtime Program.

In [ ]:
from qka.kernel_matrix import KernelMatrix
from sklearn.svm import SVC
from sklearn import metrics

# Train the SVM with the aligned kernel matrix:

kernel_aligned = result['aligned_kernel_matrix']
model = SVC(C=C, kernel='precomputed')
model.fit(X=kernel_aligned, y=y_train)

# Test the SVM on new data:

km = KernelMatrix(feature_map=fm, backend=backend)
kernel_test = km.construct_kernel_matrix(x1_vec=x_test, x2_vec=x_train, parameters=result['aligned_kernel_parameters'])
labels_test = model.predict(X=kernel_test)
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")